In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf

In [3]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Buradda kullanılan ImageGenerator kütüphanesi, resimleri toplu olarak değil de sırasıyla teker teker yükleyerek belleğin daha iyi çalışmasını sağlayacaktır.
Bu sayede ImageDataGenerator her seferinde bir resim okuyarak o resim için öz nitelik haritası çıkaracaktır.

# Veri Önişleme

## Eğitim setinin önişlenmesi

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

training_set = train_datagen.flow_from_directory('C:/Users/masla/OneDrive/Masaüstü/dataset/train',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

Found 6887 images belonging to 2 classes.


Burada resimler üzerinde kırpma işlemi uygulanarak 64x64 boyutuna indirgenir. Burada kullanılan parametreler resim işleme ile ilgili parametreleri ifade eder. Batch size'ın büyük olması, daha doğru gradyan değerinin hesaplanmasını sağlamaktadır.

### Test setinin ön işlemesi

In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = train_datagen.flow_from_directory('C:/Users/masla/OneDrive/Masaüstü/dataset/train',
                                                 target_size=(64, 64),
                                                 batch_size=32,
                                                 class_mode='binary')

Found 6887 images belonging to 2 classes.


## CNN mimarisinin oluşturulması

### Nesne oluşturulması

In [7]:
cnn = tf.keras.models.Sequential()

### 1.Adım - Convolution

In [8]:
from tensorflow.keras import layers, models

cnn = models.Sequential([
    layers.Input(shape=(64, 64, 3)),
    layers.Conv2D(filters=32, kernel_size=3, activation='relu')
])

Burada kullanılan Conv2D fonk.nu sayesindeki eldeki datalar 64x64 pixel boyutunda ele alınacaktır. Buradaki 3 parametresi ise resimler renkli olduğu için 3 adet katman oluşturulacaktır.

### 2.Adım - Pooling

In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

Pooling işlemi için MaxPool yöntemi kullanılacaktır. Pool işleminde her seferinde striders 2 olarak belirlenmiştir.

### 3.Adım - 2.Convolutional Layer

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### 4.Adım - Flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

### 5.Adım - Full Connection

In [12]:
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

Yapay sinir ağında gizli katmanlar ve nöronlar oluşturulur. Aktivasyon fonksiyonu için en sık kullanılan yöntemlerden olan relu fonksiyonu kullanılmıştır. Ne kadar katman olacağı ise units metodu ile 128 olarak belirlenmiştir. Buradaki parametre resimler 64x64 boyutunda olduğu için 128 olarak belirlenmiştir.

### 6.Adım - Çıkış Katmanı

In [13]:
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

Çıkış katmanı ise sınıflama işlemi yapılacağı için sigmoid olarak belirlenmiştir. Eğer çıktı 1 olarak çıkarsa köpek; 0 olarak çıkarsa kedi çıktısı verecektir.

### Derleme işlemi

In [14]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) 

Son adımda çıkış katmanından optimizer için adam fonksiyonu kullanmıştır. Metric olarak accuracy seçilmiştir.

### Eğitim işlemi

In [15]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [16]:
!pip install pillow

In [17]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 50)

Epoch 1/50
216/216 [==============================] - 272s 1s/step - loss: 0.6905 - accuracy: 0.5340 - val_loss: 0.6725 - val_accuracy: 0.5844
Epoch 2/50
216/216 [==============================] - 86s 396ms/step - loss: 0.6641 - accuracy: 0.6091 - val_loss: 0.6334 - val_accuracy: 0.6406
Epoch 3/50
216/216 [==============================] - 72s 335ms/step - loss: 0.6374 - accuracy: 0.6383 - val_loss: 0.6140 - val_accuracy: 0.6663
Epoch 4/50
216/216 [==============================] - 72s 333ms/step - loss: 0.6074 - accuracy: 0.6756 - val_loss: 0.5834 - val_accuracy: 0.6833
Epoch 5/50
216/216 [==============================] - 66s 306ms/step - loss: 0.5830 - accuracy: 0.6914 - val_loss: 0.5494 - val_accuracy: 0.7202
Epoch 6/50
216/216 [==============================] - 71s 331ms/step - loss: 0.5456 - accuracy: 0.7259 - val_loss: 0.5302 - val_accuracy: 0.7305
Epoch 7/50
216/216 [==============================] - 78s 361ms/step - loss: 0.5166 - accuracy: 0.7431 - val_loss: 0.4942 - val_accu

### Tahmin oluşturmak

In [45]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('C:/Users/masla/OneDrive/Masaüstü/test/kopek.jpg', target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'kopek'
else:
    prediction = 'kedi'
    

1/1 [==============================] - 0s 72ms/step


In [46]:
print(prediction)

kopek
